In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno
import re
import string
import spacy
import matplotlib.pyplot as plt
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from textwrap import wrap
import nltk
from textblob import TextBlob
from collections import Counter
from nltk.tokenize import word_tokenize

In [ ]:
def make_wordcloud(sent):
    stop_words = set(["one","little","said","the","they","that","you","in","of","and","those",
                     "or","some","when","what","where","why","how","be","to","so","as","is",
                      "am","are","was","were","this","it","on","not","no","their","all",
                     "by","upon","their","them","its","with","for","but","at","hi","there",
                     "into","from","him","his","her","such","he","she","which","an",
                     "out","then","can","up","would","if","we","about","did","my","will",
                      "very","other","could","our","thing","over","under",
                     "good","great","old","new","mr","big","small"])
    wordcloud = WordCloud(background_color="white", width=500, height=500,prefer_horizontal=1,stopwords=stop_words,
                         regexp=r"\w[\w]+").generate(sent)
    plt.figure(figsize=(10,10))
    plt.imshow(wordcloud)
    
def make_wc_from_series(df):
    noun_list = []
    for sentence in df["excerpt"]:
        #名詞
        noun_list+= TextBlob(sentence).noun_phrases
    make_wordcloud(" ".join(noun_list))

In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")

In [ ]:
train.head(3)

**id:** 一意な値 <br>
**url_legal:** 文章の取得元url(trainのみに記載)<br>
**licence:** 取得元サイトのライセンス<br>
**excerpt:** 評価対象の文 今回唯一使える特徴量<br>
**target:** 難易度を示す目的変数 値が大きいほど簡単で,小さいほど難しい<br>
**standard_error:** 難易度の標準偏差 大きいほど評価者間で難易度が分かれている

In [ ]:
a = nltk.pos_tag(nltk.word_tokenize(train.excerpt[0]))

In [ ]:
d2_list=[]
for ex in train["excerpt"]:
    a = nltk.pos_tag(nltk.word_tokenize(ex))
    c = Counter([i[1] for i in a])
    d2={}
    for k,v in c.items():
        d2[k]=pd.Series(v)
    d2_list.append(pd.DataFrame(d2))

In [ ]:
check = pd.concat(d2_list)

In [ ]:
check = check.fillna(0)

In [ ]:
#全文の長さ
train["len_sentence"] = train.excerpt.apply(lambda x: len(x))
#文の数(ピリオドの個数)
train["count_sentence"] = train.excerpt.apply(lambda x: x.count(".")+x.count("!")+x.count("?"))
#一文あたりの文の長さ
train["len_per_sentence"] = train["len_sentence"]/train["count_sentence"]
#なんとなく大文字がたくさんある文って簡単そうな気がするので大文字をカウント
train["uppers"] = train.excerpt.apply(lambda x: len(re.findall(r"[A-Z]",x)))

In [ ]:
#小文字にそろえる
train['excerpt'] = train.excerpt.apply(lambda x: x.lower())
#数字が入った単語の消去
train['excerpt']=train.excerpt.apply(lambda x: re.sub('\w*\d\w*','', x))
# []内をブランクに
train['excerpt']=train.excerpt.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
#改行の削除
train['excerpt']=train.excerpt.apply(lambda x: re.sub('\\n','', x))

In [ ]:
#単語の数
train["count_words"] = train.excerpt.apply(lambda x: len(x.split()))
#単語の平均の長さ(概算)
train["len_per_word"] = train["len_sentence"]/train["count_words"]
#名詞の数
train["count_nouns"] = train.excerpt.apply(lambda x: len(TextBlob(x).noun_phrases))
#名詞の平均の長さ
train["len_per_noun"] = train.excerpt.apply(lambda x: sum([len(y.split()[-1]) for y in TextBlob(x).noun_phrases]))/train["count_nouns"]

In [ ]:
#noun_pharasesは形容詞を含んでいる
TextBlob(train.iloc[0,3]).noun_phrases

In [ ]:
train.describe()

In [ ]:
#難易度と標準偏差に相関はない
#単語の平均の長さと若干相関がある
train.corr()

In [ ]:
#難易度の分布
sns.histplot(train["target"])

In [ ]:
#0のものが一つある
sns.histplot(train["standard_error"])

In [ ]:
print(len(train[train["standard_error"]==0]))

In [ ]:
#0を抜いて再度描画
sns.histplot(train[train["standard_error"]!=0]["standard_error"])

In [ ]:
#相関がなかったことで想像がつくが,難易度によっても別にブレの分布は変わらない
sns.histplot(train[(train["standard_error"]!=0)&(train["target"]<-1)]["standard_error"],color="r")
sns.histplot(train[(train["standard_error"]!=0)&(train["target"]>-1)]["standard_error"])

In [ ]:
#難易度の分布
sns.histplot(train["uppers"])

In [ ]:
#文の長さ
sns.histplot(train["len_sentence"])

In [ ]:
#文の数
sns.histplot(train["count_sentence"])

In [ ]:
#一文当たりの長さ(.カウントだと甘い?)
sns.histplot(train["len_per_sentence"])

In [ ]:
#単語の数
sns.histplot(train["count_words"])

In [ ]:
#一単語当たりの
sns.histplot(train["len_per_word"])

In [ ]:
#名詞の数
sns.histplot(train["count_nouns"])

In [ ]:
#一名詞当たりの長さ
sns.histplot(train["len_per_noun"])

In [ ]:
#最も簡単な評価の文
print("std:",train[train["target"]==train["target"].max()]["standard_error"].values[0])
print(train[train["target"]==train["target"].max()]["excerpt"].values[0])

恐竜が住んでいた場所について考えるとき、あなたはどんなことを思い浮かべますか？蒸し暑い沼地、鬱蒼としたジャングル、日差しの強い平原などでしょうか。恐竜はそのような場所に住んでいました。しかし、北極や南極のような寒くて暗い場所に住んでいた恐竜もいることをご存知でしょうか。古生物学者の間では、恐竜は温暖な地域にしか生息しないと考えられていました。カメやワニ、ヘビなどが生息している場所にしか、恐竜はいないと考えていたのです。その後、恐竜学者たちは意外な場所で骨を発見するようになりました。1億年前、オーストラリアは南極大陸とつながっていました。億年前のオーストラリアは、南極大陸とつながっていました。現在、古生物学者は地面から恐竜の化石を掘り出しています。彼らは、その古代の骨が何を意味するのかを考えます。

In [ ]:
make_wordcloud(train[train["target"]==train["target"].max()]["excerpt"].values[0])

In [ ]:
#最も難しい評価の文
print("std:",train[train["target"]==train["target"].min()]["standard_error"].values[0])
print(train[train["target"]==train["target"].min()]["excerpt"].values[0])

この整流子は特殊で、たった3つの銅製リングで構成されていますが、最も単純な他の連続電流発電機では、その数倍のセグメントが存在し、120個ものセグメントが見つかることもあります。この3つのセグメントは、掃除や交換のためにすぐに取り外せるように作られています。これらのセグメントは、金属製の支持体に取り付けられており、四方を自由な空気空間で囲まれているため、絶縁状態を失うことはありません。この空気絶縁の特徴は、このシステムに特有のもので、整流子の耐久性を高める上で非常に重要な要素となっています。このほか、整流子は軸のフランジに取り付けられた支持体で支えられていますが、このフランジには、絶縁体として最も優れた硬質ゴムが全面的にコーティングされています。矛盾を恐れずに言えば、これほど徹底して絶縁・保護されている整流子は他にないでしょう。3つのコンミテータは実質的に1つの銅リングを構成しており、自由な空気中に取り付けられ、その表面にあるスロットによって3等分されています。

**文章構造というよりは,文章の内容が子供たちに慣れ親しんでいるかが重要?**

In [ ]:
make_wordcloud(train[train["target"]==train["target"].min()]["excerpt"].values[0])

In [ ]:
#評価者間で難易度のブレの大きい文
print("target:",train[train["standard_error"]==train["standard_error"].max()]["target"].values[0])
print(train[train["standard_error"]==train["standard_error"].max()]["excerpt"].values[0])

前述の比較に基づいて、南西アジアの人々の間で行われている洪水の伝統の歴史を暫定的に追跡しようとする作家がいます。最近、少なくとも紀元前2000年頃に作られたバビロニアの洪水物語の断片が発見されました。この伝承は、バビロニアの歴史の初期に遡る可能性が高いと考えられます。聖書の大洪水の記述も、パレスチナではなくチグリス・ユーフラテス川流域が舞台となっています。創世記6:14-16にある箱舟の建設に関する記述は、バビロンの記述とよく似ているだけでなく、箱舟の内側と外側にアスファルトを塗りつけるという方法は、チグリス・ユーフラテス渓谷に特有のものです。そのため、多くの学者はバビロニアが聖書の洪水物語の原型であると考えています。しかし、その正確な起源はそれほど確かではありません。しかし、正確な起源は定かではありません。その詳細の多くは、有名な谷を浸水させる毎年の洪水や霧によって示唆されたものであり、バビロニアの天地創造の物語に鮮明に描かれている原始の混沌を思い起こさせるものであったことは間違いありません。また、広大な土地の地盤沈下による局所的な大洪水の記憶に基づいていたかもしれません。

In [ ]:
make_wordcloud(train[train["standard_error"]==train["standard_error"].max()]["excerpt"].values[0])

In [ ]:
#評価者間で難易度のブレがない文
print("target:",train[train["standard_error"]==train["standard_error"].min()]["target"].values[0])
print(train[train["standard_error"]==train["standard_error"].min()]["excerpt"].values[0])

曇りのない空に太陽が輝き、山には影もない。一日中、子供たちは見守っていた。ついに、鳥たちが夕暮れの星に別れの歌を歌っているとき、子供たちは影が渓谷から行進し、山の斜面を駆け上がり、ヘザーの紫を薄めているのを見た。 \そして、山の頂上が金色の槍のように輝くと、子供たちは影と太陽の間の線に目を留めました。 コンラが「さあ、その時が来たわ」と言うと、ノラが「ああ、見て見て」と言いました。そして、その扉から緑と金の服を着た小さな笛吹きが出てきました。彼が降りると、次の人、次の人と続き、全部で9人になりました。

In [ ]:
make_wordcloud(train[train["standard_error"]==train["standard_error"].min()]["excerpt"].values[0])

# アンケート全体

In [ ]:
make_wc_from_series(train)

> # target>-1

In [ ]:
make_wc_from_series(train[train["target"]>-1])

# target<=-1

In [ ]:
make_wc_from_series(train[train["target"]<=-1])

In [ ]:
from collections import defaultdict
words_count = defaultdict(int)
for sentence in train["excerpt"]:
    words = sentence.split()
    for word in words:
        words_count[word]+=1

In [ ]:
sorted_words_count = pd.DataFrame(sorted(words_count.items(),key = lambda x :x[1],reverse=True))

In [ ]:
sorted_words_count.to_csv("words_count.csv",index=False)